## Penyelesaian Sistem Persamaan Non-Linear

Sistem persamaan yang akan diselesaikan adalah:
```
f1(x, y, z) = 3x - cos(yz) - 1 = 0
f2(x, y, z) = 4y - sin(x) - z - 1 = 0
f3(x, y, z) = 5z - sqrt(x + y^2 + 1) = 0
```
Akan diselesaikan menggunakan empat metode:
1. Metode Titik Tetap (Fixed-Point Iteration)
2. Metode Newton
3. Metode Quasi-Newton (Broyden's Method)
4. Metode Steepest Descent

In [29]:
import numpy as np
import json

# --- Definisi Fungsi Sistem Persamaan F(X) ---
def F_system(X):
    x, y, z = X[0], X[1], X[2]
    f1 = 3*x - np.cos(y*z) - 1
    f2 = 4*y - np.sin(x) - z - 1
    term_sqrt = x + y**2 + 1
    if term_sqrt < 0:
      # Ini bisa terjadi selama iterasi jika jauh dari solusi.
      # Mengembalikan NaN akan menghentikan banyak metode secara alami.
      f3 = np.nan 
    else:
      f3 = 5*z - np.sqrt(term_sqrt)
    return np.array([f1, f2, f3])

# --- Definisi Jacobian J(X) ---
def Jacobian_system(X):
    x, y, z = X[0], X[1], X[2]
    J = np.zeros((3, 3))
    
    # Turunan f1
    J[0, 0] = 3  # df1/dx
    J[0, 1] = z * np.sin(y*z)  # df1/dy
    J[0, 2] = y * np.sin(y*z)  # df1/dz
    
    # Turunan f2
    J[1, 0] = -np.cos(x)  # df2/dx
    J[1, 1] = 4  # df2/dy
    J[1, 2] = -1  # df2/dz
    
    # Turunan f3
    sqrt_term_val = x + y**2 + 1
    # Pencegahan pembagian dengan nol atau akar dari negatif (meskipun F_system juga harus menangani)
    if sqrt_term_val < 1e-12: # Jika sangat kecil atau negatif
        if sqrt_term_val < 0:
            # Jika term negatif, Jacobian tidak real, return NaN atau error
            J[2,0] = np.nan
            J[2,1] = np.nan
        else: # Jika term sangat kecil (mendekati nol)
            # Gunakan regularisasi kecil untuk menghindari pembagian dengan nol
            # Atau nilai besar untuk menunjukkan turunan curam
            safe_sqrt_for_derivative = np.sqrt(sqrt_term_val + 1e-24) # Tambah epsilon kecil ke dalam akar
            J[2, 0] = -1 / (2 * safe_sqrt_for_derivative)
            J[2, 1] = -y / safe_sqrt_for_derivative
    else:
        sqrt_val = np.sqrt(sqrt_term_val)
        J[2, 0] = -1 / (2 * sqrt_val)
        J[2, 1] = -y / sqrt_val
    J[2, 2] = 5  # df3/dz
    
    return J

# --- Helper function untuk print iterasi ---
def print_iteration_details(method_name, iteration, X_current, F_X_current, step_norm=None, alpha=None, g_val=None):
    x, y, z = X_current[0], X_current[1], X_current[2]
    
    if np.any(np.isnan(F_X_current)):
        f1_val, f2_val, f3_val = np.nan, np.nan, np.nan
    else:
        f1_val, f2_val, f3_val = F_X_current[0], F_X_current[1], F_X_current[2]

    print(f"\n--- {method_name} - Iterasi {iteration} ---")
    print(f"  X = [{x:.7f}, {y:.7f}, {z:.7f}]")

    if step_norm is not None:
        print(f"  ||Step|| / ||delta_X|| / ||s_k|| = {step_norm:.3e}")
    if alpha is not None: # For Steepest Descent
        print(f"  alpha = {alpha:.3e}")
    if g_val is not None: # For Steepest Descent (objective function value)
        print(f"  g(X) = {g_val:.3e}")

    print(f"  Persamaan Saat Ini:")
    # Equation 1: 3x - cos(yz) - 1
    print(f"    f1: 3*({x:.4f}) - cos(({y:.4f})*({z:.4f})) - 1 = {f1_val:.3e}")
    # Equation 2: 4y - sin(x) - z - 1
    print(f"    f2: 4*({y:.4f}) - sin(({x:.4f})) - ({z:.4f}) - 1 = {f2_val:.3e}")
    # Equation 3: 5z - sqrt(x + y^2 + 1)
    term_under_sqrt = x + y**2 + 1
    if term_under_sqrt < 0:
        sqrt_val_display = "Error (negatif di bawah akar)"
        val_inside_sqrt_display = f"{term_under_sqrt:.4f}"
    else:
        sqrt_val_display = f"{np.sqrt(term_under_sqrt):.4f}"
        val_inside_sqrt_display = f"{term_under_sqrt:.4f}"
    print(f"    f3: 5*({z:.4f}) - sqrt({x:.4f} + ({y:.4f})^2 + 1)  [Nilai di dalam sqrt: {val_inside_sqrt_display}; Hasil sqrt: {sqrt_val_display}] = {f3_val:.3e}")
    
    if np.any(np.isnan(F_X_current)):
        print(f"  ||F(X)|| = NaN")
    else:
        print(f"  ||F(X)|| = {np.linalg.norm(F_X_current):.3e}")
    # print("-" * 40) # Optional separator

# --- Parameter Umum ---
X0 = np.array([0, 0, 0]) # Initial guess
tol = 1e-3
max_iter = 100 # Mengurangi max_iter agar output tidak terlalu panjang, bisa dinaikkan jika perlu

### 1. Metode Titik Tetap (Fixed-Point Iteration)

In [30]:
def G_fixed_point(X):
    x, y, z = X[0], X[1], X[2]
    g1 = (1 + np.cos(y*z)) / 3
    g2 = (1 + np.sin(x) + z) / 4
    term_sqrt_g3 = x + y**2 + 1
    if term_sqrt_g3 < 0:
        g3 = np.nan
    else:
        g3 = np.sqrt(term_sqrt_g3) / 5
    return np.array([g1, g2, g3])

def fixed_point_iteration(X0_fp, tol_fp, max_iter_fp):
    X = np.copy(X0_fp)
    print(f"\n=============== Metode Titik Tetap ===============")
    F_X_current = F_system(X)
    print_iteration_details("Titik Tetap", 0, X, F_X_current)
    
    for i in range(1, max_iter_fp + 1):
        X_prev = np.copy(X) # Simpan X lama untuk menghitung step_norm
        X = G_fixed_point(X_prev) # X baru adalah G(X_lama)
        
        if np.any(np.isnan(X)):
            print(f"\nIterasi {i}: NaN terdeteksi pada X. Iterasi dihentikan.")
            return X_prev, i, False # Return X_prev karena X saat ini NaN
            
        F_X_current = F_system(X)
        if np.any(np.isnan(F_X_current)):
            print_iteration_details("Titik Tetap", i, X, F_X_current, step_norm=np.linalg.norm(X - X_prev))
            print(f"\nIterasi {i}: NaN terdeteksi pada F(X). Iterasi dihentikan.")
            return X, i, False

        step_norm = np.linalg.norm(X - X_prev)
        f_norm = np.linalg.norm(F_X_current)
        
        print_iteration_details("Titik Tetap", i, X, F_X_current, step_norm=step_norm)
        
        if step_norm < tol_fp or f_norm < tol_fp:
            print(f"\nKonvergen setelah {i} iterasi.")
            print(f"Solusi: X = {X}")
            print(f"||F(X)|| = {f_norm:.3e}")
            return X, i, True
            
    print(f"\nGagal konvergen setelah {max_iter_fp} iterasi.")
    print(f"Solusi terakhir: X = {X}")
    print(f"||F(X)|| = {np.linalg.norm(F_system(X)):.3e}")
    return X, max_iter_fp, False

X_fp, iter_fp, converged_fp = fixed_point_iteration(np.copy(X0), tol, max_iter)


=============== Metode Titik Tetap ===============

--- Titik Tetap - Iterasi 0 ---
  X = [0.0000000, 0.0000000, 0.0000000]
  Persamaan Saat Ini:
    f1: 3*(0.0000) - cos((0.0000)*(0.0000)) - 1 = -2.000e+00
    f2: 4*(0.0000) - sin((0.0000)) - (0.0000) - 1 = -1.000e+00
    f3: 5*(0.0000) - sqrt(0.0000 + (0.0000)^2 + 1)  [Nilai di dalam sqrt: 1.0000; Hasil sqrt: 1.0000] = -1.000e+00
  ||F(X)|| = 2.449e+00

--- Titik Tetap - Iterasi 1 ---
  X = [0.6666667, 0.2500000, 0.2000000]
  ||Step|| / ||delta_X|| / ||s_k|| = 7.396e-01
  Persamaan Saat Ini:
    f1: 3*(0.6667) - cos((0.2500)*(0.2000)) - 1 = 1.250e-03
    f2: 4*(0.2500) - sin((0.6667)) - (0.2000) - 1 = -8.184e-01
    f3: 5*(0.2000) - sqrt(0.6667 + (0.2500)^2 + 1)  [Nilai di dalam sqrt: 1.7292; Hasil sqrt: 1.3150] = -3.150e-01
  ||F(X)|| = 8.769e-01

--- Titik Tetap - Iterasi 2 ---
  X = [0.6662501, 0.4545925, 0.2629956]
  ||Step|| / ||delta_X|| / ||s_k|| = 2.141e-01
  Persamaan Saat Ini:
    f1: 3*(0.6663) - cos((0.4546)*(0.2630)) - 

### 2. Metode Newton

In [31]:
def newton_method(X0_n, tol_n, max_iter_n):
    X = np.copy(X0_n)
    print(f"\n=============== Metode Newton ===============")
    F_val = F_system(X)
    if np.any(np.isnan(F_val)):
        print_iteration_details("Newton", 0, X, F_val)
        print("NaN terdeteksi pada F(X0). Tidak bisa memulai Newton.")
        return X, 0, False
    print_iteration_details("Newton", 0, X, F_val)
    
    for i in range(1, max_iter_n + 1):
        F_val = F_system(X) # F_k
        if np.any(np.isnan(F_val)):
            print(f"\nIterasi {i}: NaN terdeteksi pada F(X). Iterasi dihentikan.")
            return X, i, False
            
        J_val = Jacobian_system(X) # J_k
        if np.any(np.isnan(J_val)):
            print(f"\nIterasi {i}: NaN terdeteksi pada Jacobian. Iterasi dihentikan.")
            return X, i, False
        
        try:
            delta_X = np.linalg.solve(J_val, -F_val) # Solve J_k * delta_X = -F_k
        except np.linalg.LinAlgError:
            print(f"\nIterasi {i}: Jacobian singular. Iterasi dihentikan.")
            # Print state before failure
            print_iteration_details("Newton", i, X, F_val, step_norm=np.nan)
            return X, i, False

        X_new = X + delta_X # X_{k+1} = X_k + delta_X
        F_X_new = F_system(X_new)
        step_norm = np.linalg.norm(delta_X)
        
        print_iteration_details("Newton", i, X_new, F_X_new, step_norm=step_norm)
        X = X_new
        
        if np.any(np.isnan(F_X_new)):
            print(f"\nIterasi {i}: NaN terdeteksi pada F(X_new). Iterasi dihentikan.")
            return X, i, False
            
        f_norm_new = np.linalg.norm(F_X_new)
        if step_norm < tol_n or f_norm_new < tol_n:
            print(f"\nKonvergen setelah {i} iterasi.")
            print(f"Solusi: X = {X}")
            print(f"||F(X)|| = {f_norm_new:.3e}")
            return X, i, True
            
    print(f"\nGagal konvergen setelah {max_iter_n} iterasi.")
    print(f"Solusi terakhir: X = {X}")
    print(f"||F(X)|| = {np.linalg.norm(F_system(X)):.3e}")
    return X, max_iter_n, False

X_newton, iter_newton, converged_newton = newton_method(np.copy(X0), tol, max_iter)


=============== Metode Newton ===============

--- Newton - Iterasi 0 ---
  X = [0.0000000, 0.0000000, 0.0000000]
  Persamaan Saat Ini:
    f1: 3*(0.0000) - cos((0.0000)*(0.0000)) - 1 = -2.000e+00
    f2: 4*(0.0000) - sin((0.0000)) - (0.0000) - 1 = -1.000e+00
    f3: 5*(0.0000) - sqrt(0.0000 + (0.0000)^2 + 1)  [Nilai di dalam sqrt: 1.0000; Hasil sqrt: 1.0000] = -1.000e+00
  ||F(X)|| = 2.449e+00

--- Newton - Iterasi 1 ---
  X = [0.6666667, 0.4833333, 0.2666667]
  ||Step|| / ||delta_X|| / ||s_k|| = 8.655e-01
  Persamaan Saat Ini:
    f1: 3*(0.6667) - cos((0.4833)*(0.2667)) - 1 = 8.295e-03
    f2: 4*(0.4833) - sin((0.6667)) - (0.2667) - 1 = 4.830e-02
    f3: 5*(0.2667) - sqrt(0.6667 + (0.4833)^2 + 1)  [Nilai di dalam sqrt: 1.9003; Hasil sqrt: 1.3785] = -4.517e-02
  ||F(X)|| = 6.665e-02

--- Newton - Iterasi 2 ---
  X = [0.6638555, 0.4727285, 0.2747535]
  ||Step|| / ||delta_X|| / ||s_k|| = 1.363e-02
  Persamaan Saat Ini:
    f1: 3*(0.6639) - cos((0.4727)*(0.2748)) - 1 = -1.053e-05
    f2

### 3. Metode Quasi-Newton (Broyden's Method - Inverse Update)

In [32]:
def quasi_newton_broyden(X0_b, tol_b, max_iter_b):
    X = np.copy(X0_b)
    n = len(X0_b)
    print(f"\n=============== Metode Quasi-Newton (Broyden) ===============")
    
    F_current = F_system(X)
    if np.any(np.isnan(F_current)):
        print_iteration_details("Quasi-Newton (Broyden)", 0, X, F_current)
        print("NaN terdeteksi pada F(X0). Tidak bisa memulai Broyden.")
        return X, 0, False
    print_iteration_details("Quasi-Newton (Broyden)", 0, X, F_current)

    try:
        J_initial = Jacobian_system(X)
        if np.any(np.isnan(J_initial)):
            print("NaN di Jacobian awal, menggunakan matriks identitas untuk H0.")
            H = np.identity(n)
        else:
            H = np.linalg.inv(J_initial) # H0 = J(X0)^-1
    except np.linalg.LinAlgError:
        print("Jacobian awal singular, menggunakan matriks identitas untuk H0.")
        H = np.identity(n)
    
    for i in range(1, max_iter_b + 1):
        s_k = -H @ F_current # p_k = -H_k * F_k (Newton-like step)
        
        X_new = X + s_k
        F_new = F_system(X_new)
        
        step_norm = np.linalg.norm(s_k)
        print_iteration_details("Quasi-Newton (Broyden)", i, X_new, F_new, step_norm=step_norm)

        if np.any(np.isnan(F_new)):
            print(f"\nIterasi {i}: NaN terdeteksi pada F(X_new). Iterasi dihentikan.")
            return X_new, i, False

        f_norm_new = np.linalg.norm(F_new)
        if step_norm < tol_b or f_norm_new < tol_b:
            print(f"\nKonvergen setelah {i} iterasi.")
            print(f"Solusi: X = {X_new}")
            print(f"||F(X)|| = {f_norm_new:.3e}")
            return X_new, i, True
        
        y_k = F_new - F_current # delta_F = F_{k+1} - F_k
        # Broyden's "good" inverse update (Sherman-Morrison formula for H_{k+1})
        # H_{k+1} = H_k + ((s_k - H_k y_k) s_k^T H_k) / (s_k^T H_k y_k) - This is one form
        # Alternate form (often cited for inverse): H_{k+1} = H_k + ( (s_k - H_k y_k) y_k^T H_k ) / (y_k^T H_k y_k)
        # Let's use the common Sherman-Morrison update for the inverse Jacobian:
        # H_{k+1} = H_k + ( (s_k - H_k y_k) s_k^T H_k ) / (s_k^T H_k y_k)  -- THIS IS WRONG
        # Correct Sherman-Morrison for inverse: H_{k+1} = H_k + (s_k - H_k @ y_k) @ (y_k.T @ H_k) / (y_k.T @ H_k @ y_k)
        # Let's use the standard Broyden update for inverse Hessian H (approximation of J^-1):
        # H_{k+1} = H_k + ((s_k - H_k @ y_k) @ s_k.T @ H_k) / (s_k.T @ H_k @ y_k)
        # No, the most common one is: H_{k+1} = H_k + ((s_k - H_k @ y_k) @ y_k.T) / (y_k.T @ y_k)
        # Let's use the one I had before which is a known variant (often called Broyden's "good" method when updating J, and its inverse form):
        # H_{k+1} = H_k + (s_k - H_k @ y_k) @ s_k.T @ H_k / (s_k.T @ H_k @ y_k) --- denominator was s_k.T @ H_y_k

        # From Numerical Recipes / standard texts for inverse update (H approximates J^-1):
        # H_{k+1} = H_k + ( (s_k - H_k @ y_k) @ s_k.T @ H_k ) / ( s_k.T @ H_k @ y_k )
        # This is Broyden's second method or BFGS-like for J inverse.
        # The simpler one (often just called "Broyden's method" for inverse) is:
        # H_{k+1} = H_k + (s_k - H_k @ y_k) / (y_k.T @ y_k) @ y_k.T -- NO, this is for J not H.
        # For H (inverse J), it's H_new = H_old + ( (dx - H_old df) / (dx^T H_old df) ) dx^T H_old --- NO
        # H_{k+1} = H_k + ( (s_k - H_k y_k) y_k^T H_k ) / (y_k^T H_k y_k) --- This is DFP like for inverse Hessian
        # The actual Broyden method for J inverse (Sherman-Morrison on J, then Woodbury for J inverse) is:_update
        # H_new = H_old - (H_old @ y_k - s_k) @ s_k.T @ H_old / (s_k.T @ H_old @ y_k)
        # Let's use a common one: H_new = H + ( (s - H@y) @ s.T @ H ) / ( s.T @ H @ y )

        H_y_k = H @ y_k
        s_T_H = s_k.T @ H # This is a row vector
        
        # Denominator for the update of H (approximation of J^-1)
        # Using the update H_{k+1} = H_k + (s_k - H_k y_k) s_k^T H_k / (s_k^T H_k y_k)
        # This is one of Broyden's updates for the inverse.
        denominator = s_k.T @ H_y_k # This is (s_k^T H_k y_k)
        
        if abs(denominator) > 1e-12:
            numerator_term = (s_k - H_y_k) # This is a column vector
            H = H + np.outer(numerator_term, s_T_H) / denominator
        else:
            # print(f"Iterasi {i}: Denominator kecil ({denominator:.2e}) dalam update H. H tidak diupdate atau direset.")
            # Optional: reset H to identity or inverse of current Jacobian if update fails
            try:
                J_curr = Jacobian_system(X_new)
                if not np.any(np.isnan(J_curr)):
                     H = np.linalg.inv(J_curr)
                # else: H remains unchanged
            except np.linalg.LinAlgError:
                pass # H remains unchanged if Jacobian is singular

        X = X_new
        F_current = F_new
            
    print(f"\nGagal konvergen setelah {max_iter_b} iterasi.")
    print(f"Solusi terakhir: X = {X}")
    print(f"||F(X)|| = {np.linalg.norm(F_current):.3e}")
    return X, max_iter_b, False

X_broyden, iter_broyden, converged_broyden = quasi_newton_broyden(np.copy(X0), tol, max_iter)


=============== Metode Quasi-Newton (Broyden) ===============

--- Quasi-Newton (Broyden) - Iterasi 0 ---
  X = [0.0000000, 0.0000000, 0.0000000]
  Persamaan Saat Ini:
    f1: 3*(0.0000) - cos((0.0000)*(0.0000)) - 1 = -2.000e+00
    f2: 4*(0.0000) - sin((0.0000)) - (0.0000) - 1 = -1.000e+00
    f3: 5*(0.0000) - sqrt(0.0000 + (0.0000)^2 + 1)  [Nilai di dalam sqrt: 1.0000; Hasil sqrt: 1.0000] = -1.000e+00
  ||F(X)|| = 2.449e+00

--- Quasi-Newton (Broyden) - Iterasi 1 ---
  X = [0.6666667, 0.4833333, 0.2666667]
  ||Step|| / ||delta_X|| / ||s_k|| = 8.655e-01
  Persamaan Saat Ini:
    f1: 3*(0.6667) - cos((0.4833)*(0.2667)) - 1 = 8.295e-03
    f2: 4*(0.4833) - sin((0.6667)) - (0.2667) - 1 = 4.830e-02
    f3: 5*(0.2667) - sqrt(0.6667 + (0.4833)^2 + 1)  [Nilai di dalam sqrt: 1.9003; Hasil sqrt: 1.3785] = -4.517e-02
  ||F(X)|| = 6.665e-02

--- Quasi-Newton (Broyden) - Iterasi 2 ---
  X = [0.6639187, 0.4728222, 0.2753710]
  ||Step|| / ||delta_X|| / ||s_k|| = 1.392e-02
  Persamaan Saat Ini:
   

### 4. Metode Steepest Descent

In [33]:
def g_objective(X_sd):
    F_vals = F_system(X_sd)
    if np.any(np.isnan(F_vals)):
        return np.nan
    return 0.5 * np.sum(F_vals**2) # g(X) = 0.5 * ||F(X)||^2

def grad_g_objective(X_sd):
    F_val = F_system(X_sd)
    if np.any(np.isnan(F_val)):
        return np.full_like(X_sd, np.nan)
    J_val = Jacobian_system(X_sd)
    if np.any(np.isnan(J_val)):
        return np.full_like(X_sd, np.nan)
    return J_val.T @ F_val # grad(g) = J(X)^T * F(X)

def steepest_descent(X0_sd, tol_sd, max_iter_sd):
    X = np.copy(X0_sd)
    print(f"\n=============== Metode Steepest Descent ===============")
    
    alpha_init = 1.0 
    beta_ls = 0.5    
    c_ls = 1e-4      
    min_alpha_val = 1e-10 

    F_X_current = F_system(X)
    g_val_current = g_objective(X)
    if np.any(np.isnan(F_X_current)) or np.isnan(g_val_current):
        print_iteration_details("Steepest Descent", 0, X, F_X_current, g_val=g_val_current)
        print("NaN terdeteksi pada F(X0) atau g(X0). Tidak bisa memulai Steepest Descent.")
        return X, 0, False
    print_iteration_details("Steepest Descent", 0, X, F_X_current, g_val=g_val_current)

    for i in range(1, max_iter_sd + 1):
        grad_g = grad_g_objective(X)
        if np.any(np.isnan(grad_g)):
            print(f"\nIterasi {i}: NaN terdeteksi pada grad_g. Iterasi dihentikan.")
            return X,i, False
            
        grad_g_norm_sq = np.dot(grad_g, grad_g)
        
        if np.sqrt(grad_g_norm_sq) < tol_sd: 
            print(f"\nGradien sangat kecil (norm: {np.sqrt(grad_g_norm_sq):.2e}) setelah {i-1} iterasi.")
            # Cek apakah F(X) juga kecil
            f_norm_check = np.linalg.norm(F_system(X))
            if f_norm_check < tol_sd:
                 print(f"Konvergen ke solusi karena ||F(X)|| ({f_norm_check:.2e}) < toleransi.")
            else:
                 print(f"Berhenti karena gradien g(X) kecil, tetapi ||F(X)|| ({f_norm_check:.2e}) mungkin belum memenuhi toleransi (bisa jadi minimum lokal g(X) > 0).")
            break 
            
        alpha = alpha_init
        g_current_iter = g_objective(X) # g(X_k)
        
        max_line_search_iter = 50 
        for ls_iter in range(max_line_search_iter):
            X_try = X - alpha * grad_g
            g_try = g_objective(X_try)
            
            if np.isnan(g_try):
                alpha *= beta_ls # Coba alpha lebih kecil jika NaN
                if alpha < min_alpha_val:
                    break
                continue

            if g_try <= g_current_iter - c_ls * alpha * grad_g_norm_sq: # Armijo condition
                break 
            
            alpha *= beta_ls
            if alpha < min_alpha_val:
                # print(f"Iterasi {i}: Alpha terlalu kecil dalam line search.")
                break
        else: # Jika loop line search selesai tanpa break (max_line_search_iter tercapai)
            if alpha < min_alpha_val: # Dan alpha jadi sangat kecil
                print(f"\nIterasi {i}: Line search gagal menemukan alpha yang memadai. Mungkin terjebak.")
                # Cetak state terakhir sebelum break dari loop utama
                F_X_final_ls_fail = F_system(X)
                g_val_final_ls_fail = g_objective(X)
                print_iteration_details("Steepest Descent", i, X, F_X_final_ls_fail, alpha=alpha, g_val=g_val_final_ls_fail, step_norm=np.linalg.norm(alpha * grad_g))
                return X, i, False # Hentikan iterasi utama

        X_new = X - alpha * grad_g
        step_val = alpha * grad_g
        step_norm = np.linalg.norm(step_val)
        
        X = X_new
        F_X_new = F_system(X)
        g_val_new = g_objective(X)
        
        print_iteration_details("Steepest Descent", i, X, F_X_new, alpha=alpha, g_val=g_val_new, step_norm=step_norm)

        if np.any(np.isnan(F_X_new)):
            print(f"\nIterasi {i}: NaN terdeteksi pada F(X_new). Iterasi dihentikan.")
            return X, i, False

        f_norm_new = np.linalg.norm(F_X_new)
        if f_norm_new < tol_sd: # Konvergensi jika ||F(X)|| kecil
            print(f"\nKonvergen setelah {i} iterasi karena ||F(X)|| < toleransi.")
            break
        if step_norm < tol_sd * 1e-3 : # Jika step sangat kecil tapi F belum, mungkin terjebak
            # Ini kriteria tambahan untuk berhenti jika progres sangat lambat
            # print(f"\nIterasi {i}: Step sangat kecil ({step_norm:.2e}), berhenti.")
            # break 
            pass # Biarkan loop utama yang menentukan berhenti jika gradien kecil atau max_iter

    final_f_norm = np.linalg.norm(F_system(X))
    converged_status = final_f_norm < tol_sd
    if not converged_status and i == max_iter_sd : # Jika loop selesai karena max_iter
        print(f"\nGagal konvergen setelah {max_iter_sd} iterasi (mencapai max_iter).")
    # Pesan konvergensi/kegagalan lainnya sudah ditangani di dalam loop

    print(f"\nHasil akhir Steepest Descent:")
    print(f"Solusi: X = {X}")
    print(f"||F(X)|| = {final_f_norm:.3e}")
    print(f"g(X) = {g_objective(X):.3e}")
    return X, i, converged_status

X_sd, iter_sd, converged_sd = steepest_descent(np.copy(X0), tol, max_iter)


=============== Metode Steepest Descent ===============

--- Steepest Descent - Iterasi 0 ---
  X = [0.0000000, 0.0000000, 0.0000000]
  g(X) = 3.000e+00
  Persamaan Saat Ini:
    f1: 3*(0.0000) - cos((0.0000)*(0.0000)) - 1 = -2.000e+00
    f2: 4*(0.0000) - sin((0.0000)) - (0.0000) - 1 = -1.000e+00
    f3: 5*(0.0000) - sqrt(0.0000 + (0.0000)^2 + 1)  [Nilai di dalam sqrt: 1.0000; Hasil sqrt: 1.0000] = -1.000e+00
  ||F(X)|| = 2.449e+00

--- Steepest Descent - Iterasi 1 ---
  X = [0.5625000, 0.5000000, 0.5000000]
  ||Step|| / ||delta_X|| / ||s_k|| = 9.036e-01
  alpha = 1.250e-01
  g(X) = 7.057e-01
  Persamaan Saat Ini:
    f1: 3*(0.5625) - cos((0.5000)*(0.5000)) - 1 = -2.814e-01
    f2: 4*(0.5000) - sin((0.5625)) - (0.5000) - 1 = -3.330e-02
    f3: 5*(0.5000) - sqrt(0.5625 + (0.5000)^2 + 1)  [Nilai di dalam sqrt: 1.8125; Hasil sqrt: 1.3463] = 1.154e+00
  ||F(X)|| = 1.188e+00

--- Steepest Descent - Iterasi 2 ---
  X = [0.6402839, 0.5372812, 0.1395603]
  ||Step|| / ||delta_X|| / ||s_k|| = 

### Ringkasan Hasil

In [34]:
print("\n=============== Ringkasan Hasil ===============")
results_summary = {
    "Metode Titik Tetap": {"X": X_fp, "iter": iter_fp, "konvergen": converged_fp, "F_norm": np.linalg.norm(F_system(X_fp)) if not np.any(np.isnan(X_fp)) else np.nan},
    "Metode Newton": {"X": X_newton, "iter": iter_newton, "konvergen": converged_newton, "F_norm": np.linalg.norm(F_system(X_newton)) if not np.any(np.isnan(X_newton)) else np.nan},
    "Metode Quasi-Newton (Broyden)": {"X": X_broyden, "iter": iter_broyden, "konvergen": converged_broyden, "F_norm": np.linalg.norm(F_system(X_broyden)) if not np.any(np.isnan(X_broyden)) else np.nan},
    "Metode Steepest Descent": {"X": X_sd, "iter": iter_sd, "konvergen": converged_sd, "F_norm": np.linalg.norm(F_system(X_sd)) if not np.any(np.isnan(X_sd)) else np.nan}
}

for method, result in results_summary.items():
    print(f"\n{method}:")
    if result["konvergen"]:
        print(f"  Konvergen dalam {result['iter']} iterasi.")
    else:
        print(f"  Gagal konvergen atau berhenti setelah {result['iter']} iterasi.")
    print(f"  Solusi X = [{result['X'][0]:.7f}, {result['X'][1]:.7f}, {result['X'][2]:.7f}]")
    if np.isnan(result['F_norm']):
        print(f"  ||F(X)|| = NaN")
    else:
        print(f"  ||F(X)|| = {result['F_norm']:.3e}")


=============== Ringkasan Hasil ===============

Metode Titik Tetap:
  Konvergen dalam 4 iterasi.
  Solusi X = [0.6639093, 0.4725516, 0.2746220]
  ||F(X)|| = 9.053e-04

Metode Newton:
  Konvergen dalam 2 iterasi.
  Solusi X = [0.6638555, 0.4727285, 0.2747535]
  ||F(X)|| = 3.450e-05

Metode Quasi-Newton (Broyden):
  Konvergen dalam 3 iterasi.
  Solusi X = [0.6638477, 0.4727301, 0.2747735]
  ||F(X)|| = 7.677e-05

Metode Steepest Descent:
  Konvergen dalam 31 iterasi.
  Solusi X = [0.6638627, 0.4726637, 0.2749043]
  ||F(X)|| = 8.484e-04
